In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import fileio
import keys
import full_model
import data_aug
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# To create genre-specific npz files (required for randomizing splits)
#fileio.process_data_into_np_files()

# To create 27 random splits, each of size 32. (Except the last one, data/working/splits/split_26.npz is only size 5)
#fileio.create_random_splits() # Randomized with random.seed(1), so our splits will be identical

# To load a range of splits
#X, Y = fileio.load_splits(range(5)) # loads splits 0-4
#X.shape = (fileio.NUM_SAMPLES_GTZAN, 5 * 32)
#Y.shape = (24, 5 * 32)

# To generate chroma, aligned with the splits
#fileio.generate_chroma_splits_from_splits()

# To load a range of chroma splits
#X, Y = fileio.load_chroma_splits(range(5)) # loads chroma splits 0-4
#X.shape = (300, 12, 5 * 32)
#Y.shape = (24, 5 * 32)

In [3]:
def get_scores(targets, guesses):
    scores = []
    
    for idx in range(len(targets)):
        scores.append(get_score_single(targets[idx], guesses[idx, :]))
        
    return scores
    

In [4]:
def get_score_single(target, guess):
    scores = keys.get_vector_from_key(target) * 2
    return scores[keys.generate_one_hot_guess(guess.data)]

In [5]:
def load_torch_data_from_matrices(X, Y):
    X = torch.transpose(torch.from_numpy(X), 0, 2)
    Y = torch.from_numpy(keys.generate_one_hot_matrix(Y))

    X = X.float()
    Y = Y.long()
    
    return X, Y

In [6]:
import importlib

importlib.reload(full_model)

net = full_model.ShallowConvNet()
print(net)

X_test, Y_test = fileio.load_chroma_splits(range(20,27))
X_test, Y_test = load_torch_data_from_matrices(X_test, Y_test)

X, Y = fileio.load_chroma_splits(range(20))
X, Y = load_torch_data_from_matrices(X, Y)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.05)

for epoch in range(2000):
    optimizer.zero_grad()
    
    Y_hat = net(X)
    loss = criterion(Y_hat, Y)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('epoch ' + str(epoch) + ' loss: ' + str(loss.item()))
        print('Train accuracy = ' + str(np.mean(get_scores(Y, Y_hat))))
        print('Test accuracy = ' + str(np.mean(get_scores(Y_test, net(X_test)))))
        
print('Done')

ShallowConvNet(
  (conv1): Conv1d(12, 24, kernel_size=(1,), stride=(1,))
)
epoch 0 loss: 3.1772713661193848
Train accuracy = 0.097625
Test accuracy = 0.1352284263959391
epoch 100 loss: 2.7667272090911865
Train accuracy = 0.54025
Test accuracy = 0.4958375634517767
epoch 200 loss: 2.7130470275878906
Train accuracy = 0.548
Test accuracy = 0.5104568527918781


KeyboardInterrupt: 

In [ ]:
# Train accuracy
np.mean(get_scores(Y, net(X)))

In [ ]:
# Test accuracy
X, Y = fileio.load_chroma_splits(range(20,27))
X, Y = load_torch_data_from_matrices(X, Y)

np.mean(get_scores(Y, net(X)))

In [94]:
# Data aug example:
import data_aug

X, Y = fileio.load_from_range(0, 2) # Loads 2 pieces
# X.shape == [..., 2], Y.shape == [24, 2]
X_shifted, Y_shifted = data_aug.pitch_shift_all(X, Y, range(-4, 8)) # Shifts both pieces -4 to +7 semitones
# X_shifted.shape == [..., 24], Y.shape == [24, 24]